# Lambda phage

In [ ]:
from output_helper import *
%matplotlib inline

In [ ]:
with nc.Dataset("output/lp/output_t1000.nc", "r") as ds:
    grid = grid_info(ds)
    slice_vec = np.array([0, 0, 0, 0, 0])
    P_marginal = marginalDistributions(ds, grid)
    P_sliced = slicedDistributions(ds, grid, slice_vec)
    P_marginal2D = marginalDistribution2D(ds, grid)

In [ ]:
with open("scripts/reference_solutions/lp_ssa_ref.npy", "rb") as f:
    P_marginal_ssa = np.load(f, allow_pickle=True)
    P_sliced_ssa = np.load(f, allow_pickle=True)
    P_marginal2D_ssa = np.load(f, allow_pickle=True)
    nssa = np.load(f)
    nssa_min = np.load(f)
    nssa_max = np.load(f)

Parameters for Figure 5:

In [ ]:
t = 10
time_step = 1
snapshot = 100

## Figure 1
One-dimensional marginal distributions for $t = 10$

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(6, 6))
grid_ref = [range(el) + nssa_min[i] for i, el in enumerate(nssa)]
axs = plotP1Dmult(
    axs, P_marginal, P_marginal_ssa[-1], grid, grid_ref, ["DLR approx.", "SSA"], range(5), "MD")
axs.flat[0].legend()
fig.tight_layout()
fig.delaxes(axs[-1, -1])
fig.savefig('plots/lp_fig1.pgf')

## Figure 2
Sliced distributions for $x_0 = (0,\,0,\,0,\,0,\,0)$, $t = 10$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2))
grid_ref = [range(el) + nssa_min[i] for i, el in enumerate(nssa)]
axs = plotP1Dmult(
    axs, P_sliced, P_sliced_ssa[-1], grid, grid_ref, ["DLR approx.", "SSA"], range(2), "S")
axs.flat[1].legend()
fig.tight_layout()
fig.savefig('plots/lp_fig2.pgf')

## Figure 3
Two-dimensional marginal distribution of the $S_1$-$S_2$ plane for $t = 10$

Reshape `P_marginal2D` and `P_marginal2D_ssa` such that they are on the same domain:

In [ ]:
n_min = np.array([max(grid.liml[i], nssa_min[i])
                 for i in [0, 1]], dtype="int64")
n_max = np.array([min(grid.n[i] + grid.liml[i], nssa_max[i])
                  for i in [0, 1]], dtype="int64")
n = n_max - n_min
Pred = np.zeros(n[:2])
Pred_ssa = np.zeros(n[:2])

liml_red = np.array([n_min[i] - grid.liml[i]
                     for i in [0, 1]], dtype="int64")
liml_red_ssa = np.array([n_min[i] - nssa_min[i]
                         for i in [0, 1]], dtype="int64")

for i in range(n[0]):
    for j in range(n[1]):
        Pred[i, j] = P_marginal2D[i + liml_red[0], j + liml_red[1]]
        Pred_ssa[i, j] = P_marginal2D_ssa[-1][i + liml_red_ssa[0], j + liml_red_ssa[1]]

In [ ]:
xx1, xx2 = np.meshgrid(
    np.arange(n[0]) + n_min[0], np.arange(n[1]) + n_min[1], indexing='ij')
fig, ax = plotP2D(
    Pred, Pred_ssa, [xx1, xx2], ["DLR approximation", "SSA"])
plt.setp(ax.flat, xlim=[0, 8], ylim=[0, 20])
fig.tight_layout()

Maximal difference for the two-dimensional marginal distribution:

In [ ]:
print(np.amax(np.abs(Pred - Pred_ssa)))